In [130]:
#!/usr/bin/env python
# coding: utf-8

#Daniel Delgado
#On my honor, I have neither given nor received unauthorized aid on this assignment


In [131]:
import sys

In [132]:
#category 1
cat_1 = '000'
J='000'
BEQ='001'
BNE='010'
BGTZ='011'
SW='100'
LW='101'
BREAK='110'
cat1 = [cat_1,J,BEQ,BNE,BGTZ,SW,LW,BREAK]

In [133]:
#category 2
cat_2 = '001'
ADD='000'
SUB='001'
AND='010'
OR='011'
SRL='100'
SRA='101'
MUL='110'
cat2 = [cat_2,ADD,SUB,AND,OR,SRL,SRA,MUL]

In [134]:
#category 3
cat_3 = '010'
ADDI='000'
ANDI='001'
ORI='010'
cat3 = [cat_3,ADDI,ANDI,ORI]

In [135]:
cat = [cat1,cat2,cat3]

#register arrays, initialized to zero
registers = [0] * 32 #only write


data_break=260

In [136]:
#convert negative two's complement to int
def convert(val):
        tmp = val.strip()
        tmp = ''.join('1' if x == '0' else '0' for x in tmp)
        tmp = int(tmp,2) + 1
        return tmp*-1

In [137]:
#convert two's complement binary to int
def twos_complement(sample):
        if(sample[0]=='1'): #negative bit
            return convert(sample)

        elif(sample[0]=='0'): #positive bit
            return int(sample,2)

In [138]:
#read in instructions from file, return instruction array
def read_instr(file):
    input_instr = []
    f_dis = open(file,"r")
    for x in f_dis:
        input_instr.append(x)
    return input_instr

In [139]:
#read data from instr
def read_data(input_instr):
    
    address = 260
    #data registers, initialized to zero
    data = [0] * 16 #only read
    break_index = 0
    broken = False
    
    for index in range(len(input_instr)):
        instr = input_instr[index]
        input_cat = instr[:6]
        if( input_cat == cat_1 + BREAK and not broken):
            break_index = index + 1 # +! because current instruction is break, not a read in value
            broken = True
            data_break = address + 4 #with sample this should result in 316, but change otherwise
        address = address + 4
        
    for index in range(len(input_instr) - break_index-1):
        data[index] = twos_complement(input_instr[index+break_index])
    return data

In [140]:
#category 1 function
def cat_1_func(sample,break_exc):

    input_cat = sample[0:3]
    opcode = sample[3:6]

    rs = int(sample[6:11],2)
    rt = int(sample[11:16],2)

    offset = sample[16:32]
    branch_offset = offset
    offset = offset.strip() + '00'

    output = ""
    branch = 0

    if(opcode == J):
        instr_index = sample[8:]
        instr_index = instr_index.strip() + '00'
        instr_index_int = int(instr_index, 2)
        output = "J #" + str(instr_index_int)
        branch = instr_index_int

    if(opcode == BEQ):
        output = "BEQ "
        rs_string = "R" + str(rs) + ", "
        rt_string = "R" + str(rt) + ", "
        offset_string = "#" + str(int(offset,2))
        output = output + rs_string + rt_string + offset_string
        if( registers[rs] == registers[rt] ):
            branch = int(offset,2)

    if(opcode == BNE):
        output = "BNE "
        rs_string = "R" + str(rs) + ", "
        rt_string = "R" + str(rt) + ", "
        offset_string = "#" + str(int(offset,2))
        output = output + rs_string + rt_string + offset_string
        if( registers[rs] != registers[rt] ):
            branch = int(offset,2)


    if(opcode == BGTZ):
        output = "BGTZ "
        rs_string = "R" + str(rs) + ", "
        rt_string = "R" + str(rt) + ", "
        offset_string = "#" + str(int(offset,2))
        output = output + rs_string + offset_string
        if( registers[rs] > 0 ):
            branch = int(offset,2)


    if(opcode == SW): #stores into data
        #dest = rs
        #source = rt
        output = "SW "
        mem = int(sample[16:32],2)
        rs_string = "R" + str(rs)
        rt_string = "R" + str(rt) + ", "
        output = output + rt_string + str(mem) + "(" + rs_string + ")"
        data_index = int(((int(branch_offset,2)-data_break) + registers[rs])/4)
        data[data_index] = registers[rt]

    if(opcode == LW): #loads into registers
        #dest = rt
        #source = rs
        output = "LW "
        mem = int(sample[16:32],2)
        rs_string = "R" + str(rs)
        rt_string = "R" + str(rt) + ", "
        output = output + rt_string + str(mem) + "(" + rs_string + ")"
        #registers <-- from data
        data_index = int(((int(branch_offset,2)-data_break) + registers[rs])/4)
        registers[rt] = data[data_index]


    if(opcode == BREAK):
        output = "BREAK"
        break_exc = True

    #w_dis.write(output + '\n')
    return (output,break_exc,branch)

In [141]:
#category 2 function
def cat_2_func(sample):
    opcode = sample[3:6]
    rd = int(sample[6:11],2)
    rs = int(sample[11:16],2)
    rt = int(sample[16:21],2)
    output = ""


    if(opcode == ADD):
        output = "ADD "
        rd_string = "R" + str(rd) + ", "
        rs_string = "R" + str(rs) + ", "
        rt_string = "R" + str(rt)
        output = output + rd_string + rs_string + rt_string
        registers[rd] = registers[rs] + registers[rt]

    if(opcode == SUB):
        output = "SUB "
        rd_string = "R" + str(rd) + ", "
        rs_string = "R" + str(rs) + ", "
        rt_string = "R" + str(rt)
        output = output + rd_string + rs_string + rt_string
        registers[rd] = registers[rs] - registers[rt]

    if(opcode == AND):
        output = "AND "
        rd_string = "R" + str(rd) + ", "
        rs_string = "R" + str(rs) + ", "
        rt_string = "R" + str(rt)
        output = output + rd_string + rs_string + rt_string
        registers[rd] = registers[rs] and registers[rt]

    if(opcode == OR):
        output = "OR "
        rd_string = "R" + str(rd) + ", "
        rs_string = "R" + str(rs) + ", "
        rt_string = "R" + str(rt)
        output = output + rd_string + rs_string + rt_string
        registers[rd] = registers[rs] or registers[rt]

    if(opcode == SRL):
        output = "SRL "
        rd_string = "R" + str(rd) + ", "
        rs_string = "R" + str(rs) + ", "
        rt_string = "#" + str(rt)
        output = output + rd_string + rs_string + rt_string
        registers[rd] = shift_l(registers[rs], registers[rt])
        #registers[rd] = registers[rs] >> registers[rt]

        #( rd <-- rs >> rt )

    if(opcode == SRA):
        output = "SRA "
        rd_string = "R" + str(rd) + ", "
        rs_string = "R" + str(rs) + ", "
        rt_string = "#" + str(rt)
        output = output + rd_string + rs_string + rt_string
        registers[rd] = registers[rs] >> registers[rt]
        #registers[rd] = registers[rs] >> registers[rt]

        #( rd <-- rs >> rt)

    if(opcode == MUL):
        output = "MUL "
        rd_string = "R" + str(rd) + ", "
        rs_string = "R" + str(rs) + ", "
        rt_string = "R" + str(rt)
        output = output + rd_string + rs_string + rt_string
        registers[rd] = registers[rs] * registers[rt]
        #()
    return output    

In [142]:
 #category 3 function
def cat_3_func(sample):
    opcode = sample[3:6]
    rd = int(sample[6:11],2)
    rs = int(sample[11:16],2)
    imm = twos_complement(sample[16:])
    output = ""

    if(opcode == ADDI):
        output = "ADDI "
        rd_string = "R" + str(rd) + ", "
        rs_string = "R" + str(rs) + ", "
        imm_string = "#" + str(imm)
        output = output + rd_string + rs_string + imm_string
        registers[rd] = registers[rs] + imm


    if(opcode == ANDI):
        output = "ANDI "
        rd_string = "R" + str(rd) + ", "
        rs_string = "R" + str(rs) + ", "
        imm_string = "#" + str(imm)
        output = output + rd_string + rs_string + imm_string
        registers[rd] = registers[rs] and imm

    if(opcode == ORI):
        output = "ORI "
        rd_string = "R" + str(rd) + ", "
        rs_string = "R" + str(rs) + ", "
        imm_string = "#" + str(imm)
        output = output + rd_string + rs_string + imm_string
        registers[rd] = registers[rs] or imm

    return output   

In [143]:
class apple:
    def __init__(self, nuts):
        self.nuts = nuts
        
    def getnuts(self):
        return self.nuts
    
    def setnuts(self, value):
        self.nuts = value

In [144]:
#TODO:

def diassembler():
    return True

In [149]:
def main(argv):
    #read in instructions
    instr = read_instr('sample.txt')
    data = read_data(instr)
    #TODO: Write diassembler function
    
    stuff = True
    print(cat_1_func(instr[4],stuff))
    
    for i in data:
        print(i)
    
        
        
    ex = apple(10)
    print(ex.getnuts())
    
    ex.setnuts(93)
    print(ex.getnuts())
    
    


In [150]:
if __name__ == "__main__":
    main(sys.argv[1])

('LW R3, 316(R6)', True, 0)
-1
-2
-3
1
2
4
-4
10
7
9
1
0
-1
1
-1
0
10
93
